## Day 16: Ticket Translation ---
As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).  

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

class: 1-3 or 5-7  
row: 6-11 or 33-44  
seat: 13-40 or 45-50  

your ticket:  
7,1,14

nearby tickets:  
7,3,47  
40,4,50  
55,2,20  
38,6,12  
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [114]:
import numpy as np
notes_file = "data/16_notes.txt"

In [115]:
# we read the file 
with open (notes_file, "r") as file :
    info_type = 0 # 0 for instructions, 1 for your ticket, 2 for other tickets
    
#     inf_valid = []
#     sup_valid = []
    valid = {}
    tickets = []
    for line in file :
        if line == '\n' : continue
            
        if line[:12] == "your ticket:" :
            info_type = 1 
            continue
        elif line[:15] == "nearby tickets:" :
            info_type = 2
            continue
        
        if info_type == 0:
            field, rest = line.strip().split(': ')
            for inst in rest.split(' or '):
                inf, sup = inst.split('-')
#                 inf_valid.append(int(inf))
#                 sup_valid.append(int(sup))
                valid[field] = valid.get(field,[]) + [(int(inf), int(sup))]
        elif info_type == 1 :
            myticket = [int(i) for i in line.strip().split(',')]
        elif info_type == 2 :
            tickets.append([int(i) for i in line.strip().split(',')])
        else :
            print('problem for this line :')
            print(line)

In [130]:
def fus_inst (insts) :
    all_insts = []
    for inst_list in insts :
        for mini, maxi in inst_list :
            all_insts.append([mini, maxi])
    all_insts = np.array(all_insts)
    # we sort according to minimums of interval
    all_insts = all_insts[all_insts[:,0].argsort()]

    fused_inst = all_insts[0:1,:]
    for mini, maxi in all_insts[1:,:] :
        if mini > fused_inst[-1,1]+1 :
            fused_inst = np.append(fused_inst, [[mini, maxi]], 0)
        else :
            fused_inst[-1,1] = maxi
    return fused_inst

def check_in_insts(n, insts) :
    i_small = 0
    i_big = len(insts) - 1
    
    if i_big == 0 :
        return insts[0,0] <= n <= insts[0,1]
    
    while i_small < i_big :
        i = (i_small + i_big) // 2
#         print(i_small, i_big, i)
        if n < insts[i,0] :
            i_big = i
        else :
            i_small = max(i, i_small + 1)
#     print(i_small, i_big, i)

    if n < insts[i+1,0] :
        return n <= insts[i,1] and n >= insts[i,0]
    return  n <= insts[i+1,1]

insts = fus_inst(valid.values())
tot = 0
for t in tickets :
    for val in t :
        if not check_in_insts(val,insts) :
            tot += val 
tot

32835

## Part Two ---
Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

class: 0-1 or 4-19  
row: 0-5 or 8-19  
seat: 0-13 or 16-19  

your ticket:  
11,12,13  

nearby tickets:  
3,9,18  
15,1,5  
5,14,9  
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.  

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?  

In [169]:
notes_file = "data/16_notes.txt"
# we read the file 
with open (notes_file, "r") as file :
    info_type = 0 # 0 for instructions, 1 for your ticket, 2 for other tickets
    
#     inf_valid = []
#     sup_valid = []
    valid = {}
    tickets = []
    for line in file :
        if line == '\n' : continue
            
        if line[:12] == "your ticket:" :
            info_type = 1 
            continue
        elif line[:15] == "nearby tickets:" :
            info_type = 2
            continue
        
        if info_type == 0:
            field, rest = line.strip().split(': ')
            for inst in rest.split(' or '):
                inf, sup = inst.split('-')
#                 inf_valid.append(int(inf))
#                 sup_valid.append(int(sup))
                valid[field] = valid.get(field,[]) + [(int(inf), int(sup))]
        elif info_type == 1 :
            myticket = [int(i) for i in line.strip().split(',')]
        elif info_type == 2 :
            tickets.append([int(i) for i in line.strip().split(',')])
        else :
            print('problem for this line :')
            print(line)

In [189]:
insts = fus_inst(valid.values())
valid = {k: fus_inst([v]) for k,v in valid.items()}
tot = 0

# possible_field = {k : list(range(len(tickets[0]))) for k in valid.keys()}
possible_class = [list(valid.keys()) for _ in range(len(tickets[0]))]

for t in tickets :
#     print(t)
    skip_t = False
    for val in t :
        skip_t |= not check_in_insts(val,insts) 
#     print(skip_t)
    if skip_t :
        continue
    for i, val in enumerate(t):
        keep = []
        for c in possible_class[i] :
            if check_in_insts(val,valid[c]) :
                keep.append(c)
        possible_class[i] = keep
#     break

keep_going = True
while keep_going :
    keep_going = False
    
    for i in range(len(tickets[0])) :
        if len(possible_class[i]) > 1 :
            keep_going = True
        else :
            c = possible_class[i][0]
            for j, pc in enumerate(possible_class) :
                if j != i :
                    if c in pc :
                        pc.remove(c)
        


In [196]:
tot = 1 
for i,c in enumerate(possible_class) :
    c = c[0]
    if c[:len("departure")] == "departure" :
        tot *= myticket[i]
tot

514662805187